In [6]:
import time
start = time.time()
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


In [7]:
df_train = pd.read_csv("/Users/abishekbhuvanaratnam/Documents/first-ds-project/first-ds-project/data/raw/train.csv")